In [ ]:
!git clone https://github.com/fabianagoes/ismb_tutorial8.git

In [ ]:
%cd ismb_tutorial8

In [ ]:
!pip uninstall -y numpy scipy gensim
!pip install numpy==1.24.4 scipy==1.10.1 gensim==4.3.3

In [ ]:
# 3. Importações
import os
from gensim.models import Word2Vec
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 4. Função para carregar sequências
def load_sequences(filepath):
    sequences = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip().upper()
            if line and not line.startswith('>'):
                sequences.append(line)
    return sequences

# 5. Função para gerar k-mers
def kmerize(sequence, k=3):
    return [sequence[i:i+k] for i in range(len(sequence)-k+1)]


In [ ]:
root_dir = "/content/ismb_tutorial8/datasets"
subdirs = get_all_subdirs(root_dir) # internal function that goes over all subdirectories. Source code: utils.py

dropdown = widgets.Dropdown(
    options=subdirs,
    description='Select:',
    disabled=False,
)

selected_path = {'value': subdirs[0]}

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_path['value'] = change['new']

dropdown.observe(on_change)

display(dropdown)

In [ ]:
print('Dataset Selected: ',selected_path['value'])

train_data=pd.read_csv(selected_path['value']+'/train.csv')#read_csv to read csv files
val_data=pd.read_csv(selected_path['value']+'/dev.csv')
test_data=pd.read_csv(selected_path['value']+'/test.csv')
data={'Train':train_data,'Val':val_data,'Test':test_data}

train_data.head(5)# displays first 5 datapoints

In [ ]:
# Função para quebrar sequência em k-mers
def kmerize(sequence, k):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# Criar corpus para word2vec usando as sequências do conjunto de treino
k = 3
corpus = [kmerize(seq, k) for seq in train_data['sequence']]

# Treinar o modelo word2vec
model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=2, epochs=10)

# Mostrar vetor do primeiro k-mer da primeira sequência
example_kmer = corpus[0][0]
print(f"Vetor embedding para {example_kmer}:")
print(model.wv[example_kmer])

# Mostrar k-mers mais similares
print("K-mers mais similares a", example_kmer)
print(model.wv.most_similar(example_kmer))
